# Setup

In [1]:
import numpy as np
import os
import glob
import pandas as pd
import sys
import yaml

## Process Config

In [2]:
with open( './config.yml', "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
input_dir = os.path.join( config['data_dir'], config['input_dirname'] )

In [4]:
data_fp = os.path.join( input_dir, config['website_data_file_pattern'] )
data_fp = glob.glob( data_fp )[0]

In [5]:
press_office_data_fp = os.path.join( input_dir, config['press_office_data_file_pattern'] )
press_office_data_fp = glob.glob( press_office_data_fp )[0]

In [6]:
output_dir = os.path.join( config['data_dir'], config['output_dirname'] )

In [7]:
grouping_labels = [ group_by_i.lower().replace( ' ', '_' ) for group_by_i in config['groupings'] ]

# Transform Website Data

## Preprocessing

In [8]:
# Load data
df = pd.read_csv( data_fp, parse_dates=[ 'Date', ] )

In [9]:
# Drop drafts
df.drop( df.index[df['Date'].dt.year == 1970], axis='rows', inplace=True )

In [10]:
# Drop weird articles---ancient ones w/o a title or press type
df.dropna( axis='rows', how='any', subset=[ 'Title', 'Press Types', ], inplace=True )

In [11]:
# Get rid of HTML ampersands
for str_column in [ 'Title', 'Research Topics', 'Categories' ]:
    df[str_column] = df[str_column].str.replace( '&amp;', '&' )

In [12]:
# Get date bins
start_year = df['Date'].min().year - 1
end_year = df['Date'].max().year + 1
date_bins = pd.date_range(
    '{} {}'.format( config['year_start'], start_year ),
    pd.Timestamp.now() + pd.offsets.DateOffset( years=1 ),
    freq = pd.offsets.DateOffset( years=1 ),
)
date_bin_labels = date_bins.year[:-1]

In [13]:
# Add the year published (using the above start date )
df['Year'] = pd.cut( df['Date'], date_bins, labels=date_bin_labels ) 

## Grouped Counts

In [14]:
for i, group_by_i in enumerate( config['groupings'] ):
    df_i = df.copy()

    # Explode and group
    df_i[group_by_i] = df_i[group_by_i].str.split( '|' )
    df_i = df_i.explode( group_by_i )

    # Get counts
    counts = df_i.pivot_table( index='Year', columns=group_by_i, values='id', aggfunc='count' )
    
    # Save
    output_fn = 'counts.{}.csv'.format( grouping_labels[i] )
    counts_output_dir = os.path.join( output_dir, 'counts' )
    os.makedirs( counts_output_dir, exist_ok=True )
    output_fp = os.path.join( counts_output_dir, output_fn )
    counts.to_csv( output_fp, )

# Coordinate Manual Data

In [15]:
# Load press data
press_df = pd.read_excel( press_office_data_fp )
press_df.set_index( 'id', inplace=True )
press_df.drop( 'Title (optional)', axis='columns', inplace=True )
for column in [ 'Press Mentions', 'People Reached' ]:
    press_df[column] = press_df[column].astype( 'Int64' )

In [16]:
# Combine with website data
combined_df = df.set_index( 'id' ).join( press_df )

In [17]:
# Save combined data
merged_output_dir = os.path.join(  output_dir, 'merged' )
os.makedirs( merged_output_dir, exist_ok=True )
output_fp = os.path.join( merged_output_dir, os.path.basename( data_fp ) )
combined_df.to_csv( output_fp )